In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline
best_model = None
best_score = 0
hospital_name = 'host1'

In [2]:
data = pd.read_csv('../data/hospital' + str(1) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.004
[MICE] Starting imputation round 2/110, elapsed time 0.033
[MICE] Starting imputation round 3/110, elapsed time 0.056
[MICE] Starting imputation round 4/110, elapsed time 0.079
[MICE] Starting imputation round 5/110, elapsed time 0.102
[MICE] Starting imputation round 6/110, elapsed time 0.125
[MICE] Starting imputation round 7/110, elapsed time 0.148
[MICE] Starting imputation round 8/110, elapsed time 0.171
[MICE] Starting imputation round 9/110, elapsed time 0.193
[MICE] Starting imputation round 10/110, elapsed time 0.230
[MICE] Starting imputation round 11/110, elapsed time 0.258
[MICE] Starting imputation round 12/110, elapsed time 0.281
[MICE] Starting imputation round 13/110, elapsed time 0.304
[MICE] Starting imputation round 14/110, elapsed time 0.327
[MICE] Starting imputation round 15/110, elapsed time 0.349
[MICE] Starting imputation round 16/110, elapsed time 0.372
[MI

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'f1'

In [6]:
balancer = SMOTE(random_state=42)

In [7]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [8]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [9]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pre

In [10]:
gs_dummy.best_params_

{'clf__strategy': 'stratified'}

In [11]:
gs_dummy.best_score_

0.39645732689210955

In [12]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', rf))
pipeline = Pipeline(plist)

gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [13]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  2.0min finished


In [14]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'entropy',
 'clf__max_depth': 10,
 'clf__n_estimators': 100,
 'clf__random_state': 42}

In [15]:
gs_rf.best_score_


0.46700112110862624

In [16]:
if gs_rf.best_score_ > best_score:
    best_model = gs_rf
    best_score = gs_rf.best_score_

In [17]:
show_f_importance(gs_rf.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 45 - Parea_ECG_V8 (0.029742)
2. feature 46 - Parea_ECG_V9 (0.029327)
3. feature 1 - QTm_old (0.028393)
4. feature 53 - Pfwhm_ECG_V1 (0.027602)
5. feature 66 - patsex (0.027417)
6. feature 40 - Parea_ECG_V3R (0.025721)
7. feature 48 - Parea_ECG_aVL (0.023909)
8. feature 34 - Parea_ECG_I (0.022841)
9. feature 3 - QTn (0.022701)
10. feature 56 - Pfwhm_ECG_V3R (0.022539)
11. feature 43 - Parea_ECG_V5 (0.021343)
12. feature 41 - Parea_ECG_V4 (0.019882)
13. feature 67 - Age (0.019763)
14. feature 58 - Pfwhm_ECG_V4R (0.019106)
15. feature 36 - Parea_ECG_III (0.018968)
16. feature 55 - Pfwhm_ECG_V3 (0.018048)
17. feature 65 - Pfwhm_ECG_aVR (0.017986)
18. feature 59 - Pfwhm_ECG_V5 (0.017255)
19. feature 21 - Pdur_ECG_V1 (0.016202)
20. feature 17 - Pdur16iqr (0.016162)
21. feature 16 - Pdur16std (0.015909)
22. feature 63 - Pfwhm_ECG_aVF (0.015360)
23. feature 44 - Parea_ECG_V6 (0.015339)
24. feature 30 - Pdur_ECG_V9 (0.015181)
25. feature 18 - Pdur_ECG_I (0.014997)
26

In [18]:
param_grid = {"clf__max_depth": [3,5,8],
              "clf__n_estimators": [100,200,500,1000],
              "clf__min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
# 

gb = GradientBoostingClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', gb))
pipeline = Pipeline(plist)
gs_gb = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [19]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.1min finished


In [20]:
gs_gb.best_params_

{'clf__max_depth': 3, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 200}

In [21]:
gs_gb.best_score_

0.43485860244480934

In [22]:
if gs_gb.best_score_ > best_score:
    best_model = gs_gb
    best_score = gs_gb.best_score_

In [23]:
show_f_importance(gs_gb.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 67 - Age (0.047531)
2. feature 40 - Parea_ECG_V3R (0.036437)
3. feature 34 - Parea_ECG_I (0.034807)
4. feature 1 - QTm_old (0.034795)
5. feature 0 - RR (0.034542)
6. feature 3 - QTn (0.034459)
7. feature 53 - Pfwhm_ECG_V1 (0.030270)
8. feature 21 - Pdur_ECG_V1 (0.028328)
9. feature 50 - Pfwhm_ECG_I (0.026645)
10. feature 59 - Pfwhm_ECG_V5 (0.025919)
11. feature 46 - Parea_ECG_V9 (0.025903)
12. feature 66 - patsex (0.025656)
13. feature 10 - PCarea (0.025576)
14. feature 64 - Pfwhm_ECG_aVL (0.025229)
15. feature 39 - Parea_ECG_V3 (0.025185)
16. feature 45 - Parea_ECG_V8 (0.023996)
17. feature 54 - Pfwhm_ECG_V2 (0.023633)
18. feature 58 - Pfwhm_ECG_V4R (0.023553)
19. feature 2 - QR (0.021974)
20. feature 42 - Parea_ECG_V4R (0.021241)
21. feature 36 - Parea_ECG_III (0.021191)
22. feature 38 - Parea_ECG_V2 (0.020172)
23. feature 16 - Pdur16std (0.019473)
24. feature 65 - Pfwhm_ECG_aVR (0.019442)
25. feature 48 - Parea_ECG_aVL (0.018271)
26. feature 55 - Pfwhm_EC

In [24]:
param_grid = {"clf__learning_rate": [0.1,0.3,0.7,0.8],
              "clf__n_estimators": [100,1000,2000,2500,3000,5000]}

ab = AdaBoostClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', ab))
pipeline = Pipeline(plist)

gs_ab = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [25]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  6.3min finished


In [26]:
gs_ab.best_params_

{'clf__learning_rate': 0.3, 'clf__n_estimators': 3000}

In [27]:
gs_ab.best_score_

0.43961434023543317

In [28]:

if gs_ab.best_score_ > best_score:
    best_model = gs_ab
    best_score = gs_ab.best_score_

In [29]:
show_f_importance(gs_ab.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 21 - Pdur_ECG_V1 (0.043667)
2. feature 14 - Pdur16min (0.037333)
3. feature 34 - Parea_ECG_I (0.036333)
4. feature 57 - Pfwhm_ECG_V4 (0.032000)
5. feature 10 - PCarea (0.030667)
6. feature 46 - Parea_ECG_V9 (0.030000)
7. feature 1 - QTm_old (0.029333)
8. feature 40 - Parea_ECG_V3R (0.029333)
9. feature 33 - Pdur_ECG_aVR (0.028000)
10. feature 39 - Parea_ECG_V3 (0.027000)
11. feature 51 - Pfwhm_ECG_II (0.025667)
12. feature 67 - Age (0.025000)
13. feature 52 - Pfwhm_ECG_III (0.024667)
14. feature 43 - Parea_ECG_V5 (0.024333)
15. feature 16 - Pdur16std (0.024000)
16. feature 63 - Pfwhm_ECG_aVF (0.023667)
17. feature 45 - Parea_ECG_V8 (0.023333)
18. feature 0 - RR (0.023000)
19. feature 36 - Parea_ECG_III (0.022667)
20. feature 65 - Pfwhm_ECG_aVR (0.022000)
21. feature 3 - QTn (0.020667)
22. feature 58 - Pfwhm_ECG_V4R (0.020333)
23. feature 53 - Pfwhm_ECG_V1 (0.020000)
24. feature 27 - Pdur_ECG_V5 (0.019000)
25. feature 48 - Parea_ECG_aVL (0.018667)
26. feature

In [30]:
param_grid = {"clf__max_depth": [3,5,8,10,15,30],
              "clf__n_estimators": [100,200,500,1000,1500],
              "clf__min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', et))
pipeline = Pipeline(plist)

gs_et = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [31]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.6min finished


In [32]:
gs_et.best_params_

{'clf__max_depth': 3, 'clf__min_samples_leaf': 5, 'clf__n_estimators': 1000}

In [33]:
gs_et.best_score_

0.4845598434487323

In [34]:
if gs_et.best_score_ > best_score:
    best_model = gs_et
    best_score = gs_et.best_score_

In [35]:
show_f_importance(gs_et.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.097806)
2. feature 45 - Parea_ECG_V8 (0.067732)
3. feature 46 - Parea_ECG_V9 (0.066875)
4. feature 53 - Pfwhm_ECG_V1 (0.047826)
5. feature 67 - Age (0.044778)
6. feature 1 - QTm_old (0.032894)
7. feature 59 - Pfwhm_ECG_V5 (0.028630)
8. feature 55 - Pfwhm_ECG_V3 (0.026778)
9. feature 36 - Parea_ECG_III (0.025097)
10. feature 34 - Parea_ECG_I (0.023361)
11. feature 58 - Pfwhm_ECG_V4R (0.019812)
12. feature 54 - Pfwhm_ECG_V2 (0.019494)
13. feature 60 - Pfwhm_ECG_V6 (0.018900)
14. feature 39 - Parea_ECG_V3 (0.018274)
15. feature 44 - Parea_ECG_V6 (0.018165)
16. feature 9 - PCfwhm (0.018140)
17. feature 48 - Parea_ECG_aVL (0.016595)
18. feature 3 - QTn (0.016209)
19. feature 49 - Parea_ECG_aVR (0.016124)
20. feature 56 - Pfwhm_ECG_V3R (0.015881)
21. feature 35 - Parea_ECG_II (0.014404)
22. feature 40 - Parea_ECG_V3R (0.013273)
23. feature 57 - Pfwhm_ECG_V4 (0.013215)
24. feature 0 - RR (0.012981)
25. feature 10 - PCarea (0.012636)
26. feature 61 - 

In [36]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [37]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.002
[MICE] Starting imputation round 2/110, elapsed time 0.030
[MICE] Starting imputation round 3/110, elapsed time 0.054
[MICE] Starting imputation round 4/110, elapsed time 0.077
[MICE] Starting imputation round 5/110, elapsed time 0.100
[MICE] Starting imputation round 6/110, elapsed time 0.123
[MICE] Starting imputation round 7/110, elapsed time 0.149
[MICE] Starting imputation round 8/110, elapsed time 0.181
[MICE] Starting imputation round 9/110, elapsed time 0.211
[MICE] Starting imputation round 10/110, elapsed time 0.238
[MICE] Starting imputation round 11/110, elapsed time 0.263
[MICE] Starting imputation round 12/110, elapsed time 0.290
[MICE] Starting imputation round 13/110, elapsed time 0.313
[MICE] Starting imputation round 14/110, elapsed time 0.337
[MICE] Starting imputation round 15/110, elapsed time 0.363
[MICE] Starting imputation round 16/110, elapsed time 0.399
[MI

In [38]:
from sklearn.metrics import roc_auc_score
p = best_model.predict(cx_df);
scores = pd.DataFrame({hospital_name: p})
scores.to_csv('../data/' + hospital_name +'_predictions.csv', index=False)
np.save('../data/' + hospital_name +'_acc',best_score)
roc_auc_score(cy_df, p)

0.5589787073676276

In [39]:
best_model

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('balancer', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=42, ratio='auto', svm_estimator=None)), ('clf', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', ...estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [3, 5, 8, 10, 15, 30], 'clf__n_estimators': [100, 200, 500, 1000, 1500], 'clf__min_samples_leaf': [1, 3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=1)

In [40]:
best_score

0.4845598434487323

In [41]:
from sklearn.externals import joblib
joblib.dump(best_model.best_estimator_, '../data/' + hospital_name + '_model.pkl')

['../data/host1_model.pkl']